# **ArticlAide**

An agents to Research and Write an Article

- LLM
- Agent
- Task
- Crew

### **Install Required Packages**

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_google_genai
%pip install --quiet -U crewai

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### **Import Required Libraries**

In [ ]:
from crewai import Agent, Task, Crew, LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import os
from IPython.display import Markdown
from IPython.display import  display
import textwrap


### **Set Up API Key**

In [ ]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

### **Initialize an LLM**

In [ ]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.5,
)

### **Create Agents**

Define Agents, and provide them a `role`, `goal` and `backstory`.

In [ ]:
# Agent: Planner

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [ ]:
# Agent: Writer

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

In [ ]:
# Agent: Editor

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
)

### **Create Tasks**

In [ ]:
# Task: Plan

plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [ ]:
# Task: Write

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [ ]:
# Task: Edit

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### **Creating the Crew**

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

### **Running the Crew and Displaying Response**

In [ ]:
def to_markdown(text):
    # Replace bullet points with asterisks for markdown formatting
    text = text.replace("•", "  *")

    # Indent the text with a '>' character for blockquote style
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan: Understanding the Rise of Artificial Intelligence**

**1. Prioritized Trends, Key Players, and Noteworthy News:**

* **Trend:** Generative AI –  Focus on advancements in models like GPT-4, DALL-E 2, and Stable Diffusion.  Highlight their capabilities and limitations, including ethical concerns around bias and misinformation.
* **Trend:** AI in Healthcare – Discuss AI's role in drug discovery, personalized medicine, diagnostics, and robotic surgery.  Mention key players like IBM Watson Health and Google DeepMind.  Noteworthy news could include recent FDA approvals of AI-powe

In [ ]:
to_markdown(result.raw)

> # Understanding the Rise of Artificial Intelligence: A Look at Trends, Players, and the Future
> 
> **Introduction: The AI Revolution is Here**
> 
> Artificial intelligence (AI) is rapidly transforming our world.  Its impact spans numerous sectors, from self-driving cars to medical diagnoses.  Recent breakthroughs, such as AlphaFold's accurate protein structure prediction, highlight AI's potential to revolutionize various fields, including drug discovery. This article explores current AI trends, key players, ethical considerations, and the technology's potential future impact. We will examine different AI types, including narrow or weak AI (designed for specific tasks) and general or strong AI (a hypothetical AI with human-level intelligence).  The discussion aims to provide a balanced overview of this rapidly evolving field.
> 
> 
> **Key Trends in Artificial Intelligence: A Multifaceted Landscape**
> 
> **A. Generative AI: Creating New Worlds**
> 
> Generative AI, a subset of AI focused on creating new content, is experiencing significant growth.  Models like GPT-4 and DALL-E 2 generate human-quality text, images, and code. While these advancements are impressive, limitations exist. For instance, GPT-4 can sometimes produce inaccurate information, and DALL-E 2 may struggle with complex visual scenes.  Ethical concerns, such as the potential for misuse in creating deepfakes and spreading misinformation, require careful consideration.  [Insert image of AI-generated art here]
> 
> 
> **B. AI in Healthcare: Revolutionizing Medicine**
> 
> AI's applications in healthcare are vast and transformative.  From accelerating drug discovery to improving diagnostic accuracy, AI is enhancing medical practices.  Examples include IBM Watson Health's use of AI to analyze medical data and Google DeepMind's AlphaFold protein structure prediction.  AI-guided robotic surgery increases precision and minimizes invasiveness.  The increasing FDA approval of AI-powered medical devices reflects growing acceptance and integration within clinical settings.  The potential for personalized medicine based on individual genetic profiles is particularly promising.
> 
> 
> **C. AI Across Industries: A Widespread Impact**
> 
> AI's transformative influence extends beyond healthcare.  In finance, AI aids in fraud detection and algorithmic trading.  Manufacturing benefits from AI-powered automation and predictive maintenance.  Self-driving vehicles are reshaping the transportation sector.  Even customer service increasingly relies on AI-powered chatbots.  The applications are diverse and continue to expand rapidly.
> 
> 
> **D. AI Ethics and Regulation: Navigating the Moral Maze**
> 
> The rapid advancement of AI necessitates careful consideration of ethical and regulatory aspects.  Algorithmic bias, where AI systems perpetuate societal biases, is a significant concern.  Automation-driven job displacement and privacy concerns related to data collection and use also require attention.  The EU's AI Act represents a significant step towards addressing these issues, but global collaboration is crucial for responsible AI development and deployment.
> 
> 
> **Key Players in the AI Landscape: The Titans of Tech**
> 
> Several companies are leading AI innovation.  OpenAI, with its language models like GPT, is a key player.  Google's DeepMind pushes the boundaries of AI research.  Microsoft integrates AI into its products and services. Amazon's AWS provides cloud infrastructure for many AI applications.  Meta (formerly Facebook) focuses on computer vision and natural language processing.  NVIDIA's GPUs are essential for training complex AI models. [Include logos of the mentioned companies]
> 
> 
> **The Future of Artificial Intelligence: A Glimpse into Tomorrow**
> 
> The future of AI holds both exciting possibilities and challenges.  We can anticipate further advancements in generative AI, leading to even more realistic and creative content.  AI's role in addressing global challenges, such as climate change and disease, is expected to grow.  However, potential misuse and ethical dilemmas necessitate careful consideration and proactive regulation.  The long-term economic and societal impact will depend heavily on responsible development and deployment strategies.
> 
> 
> **Conclusion: Embracing Responsibility**
> 
> Artificial intelligence is transforming our world at an unprecedented rate.  While its potential benefits are substantial, so are the risks.  Generative AI, AI in healthcare, and widespread industrial adoption are reshaping our lives.  Responsible development and ethical considerations are paramount.  We must proactively address algorithmic bias, job displacement, and privacy concerns.  The future of AI hinges on our collective commitment to harnessing its power for the benefit of all humanity.
> 
> 
> **Call to Action: Join the Conversation**
> 
> To learn more about AI, explore resources like the MIT Technology Review and Wired.  You can also visit the websites of the companies mentioned above.  Share your thoughts on the ethical considerations of AI in the comments section below. Let's engage in a constructive dialogue to shape a future where AI serves humanity.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence and sports"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence and sports.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan: Artificial Intelligence Revolutionizing Sports**

**1. Prioritized Trends, Key Players, and News:**

* **Trend:** AI-powered performance analysis is moving beyond simple metrics to predictive modeling of injury risk, optimal training regimens, and personalized player development strategies.  This includes sophisticated biomechanical analysis using computer vision.
* **Trend:**  AI is enhancing fan experience through personalized content recommendations, interactive virtual experiences, and improved accessibility features (e.g., real-time translation of commentary

In [ ]:
to_markdown(result.raw)

> # AI: Revolutionizing the World of Sports
> 
> Artificial intelligence (AI) is rapidly transforming the sports world, impacting how games are played, coached, and experienced.  From optimizing player performance to enhancing fan engagement and improving officiating, AI's influence is undeniable. This article explores AI's multifaceted impact on sports, examining its applications in performance analysis, fan engagement, and officiating, while also addressing crucial ethical considerations. We will also look at key players and trends shaping this rapidly evolving landscape.
> 
> 
> ## AI-Powered Performance Analysis: Data-Driven Athlete Development
> 
> AI is revolutionizing player performance analysis.  Advanced algorithms analyze data from various sources—video footage, wearable sensors, GPS tracking—providing a nuanced understanding of player capabilities. Computer vision, for example, analyzes movement patterns, identifying inefficiencies for targeted training improvements.  This detailed analysis goes beyond basic statistics, offering a more comprehensive assessment of player potential.
> 
> Machine learning models are also proving invaluable in injury prediction and prevention. By identifying patterns and risk factors in player data, AI helps teams proactively address potential issues, optimizing training and reducing injury risk. This personalized, AI-driven approach allows coaches to tailor training programs to individual needs, maximizing performance while minimizing injury.  Many professional sports teams are already utilizing this technology to gain a competitive edge.
> 
> 
> ## Enhancing the Fan Experience Through AI-Powered Engagement
> 
> AI is not only transforming on-field performance but also the fan experience.  AI algorithms deliver personalized content recommendations, ensuring fans receive tailored updates and highlights. This customization fosters stronger connections between fans and their teams.  Beyond personalized content, AI is creating immersive virtual experiences.  Augmented reality (AR) applications, for example, offer fans exciting new ways to interact with the game, bridging the physical and digital worlds.
> 
> AI is also enhancing accessibility for fans with disabilities.  Real-time translation of commentary and descriptive audio for visually impaired fans are examples of how AI promotes inclusivity in sports. The potential for AI to enhance fan engagement is vast, promising even more innovative strategies in the future.
> 
> 
> ## AI in Officiating: Striving for Accuracy and Consistency
> 
> AI's application in officiating is both exciting and complex.  Computer vision and machine learning algorithms offer the potential to significantly improve the accuracy and consistency of calls, particularly in high-speed sports where split-second decisions are crucial.  AI systems can detect offside positions, fouls, and other infractions, providing referees with valuable support.
> 
> However, challenges remain. Concerns about algorithmic bias and the need for human oversight are paramount.  Ensuring AI systems are trained on diverse and representative datasets is crucial to mitigate bias.  A balanced approach, combining human judgment with AI's precision, is likely the most effective path forward. While fully automated officiating is still some way off, AI's role as a supportive tool for referees is rapidly expanding.
> 
> 
> ## Ethical Considerations: Responsible AI in Sports
> 
> The integration of AI in sports raises several ethical considerations. Data privacy is a major concern, requiring careful attention to consent and data security.  Algorithmic bias, as discussed earlier, needs careful management.  The potential displacement of human jobs due to automation also requires proactive solutions.
> 
> A responsible approach is crucial to ensure AI is used ethically and equitably. Open discussions involving athletes, coaches, officials, and fans are necessary to establish guidelines and best practices.  The human element of sports—passion, emotion, and unpredictability—should be preserved alongside technological advancements.  Finding a balance between innovation and ethical considerations is essential for the future of AI in sports.
> 
> 
> ## Conclusion:  A Transformative Technology with Ethical Responsibilities
> 
> AI is revolutionizing the sports industry, offering opportunities to improve player performance, enhance fan experiences, and improve officiating accuracy.  However, addressing the ethical challenges associated with AI's growing role is crucial.  By proactively addressing these concerns and fostering open dialogue, we can harness AI's potential while preserving the integrity and human spirit of sports.  Staying informed about AI advancements and actively participating in discussions about ethical implications is vital as AI shapes the future of sports.

---